<a href="https://colab.research.google.com/github/NadaTarekk/Courses-Recommandion/blob/main/Courses_Recommandtion_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np

In [48]:
courses_data = pd.read_csv('/content/Courses.csv')


In [49]:
courses_data.head()

,Unnamed: 0,instructors,course_title,rating,numOfstudents,descriptions,languages,lastUpdated,link,level,category,numOfratings
0,0,['Jose Portilla'],The Complete Python Bootcamp From Zero to Hero...,4.6,"1,823,671 students",Learn Python like a Professional Start from t...,English,Last updated 7/2023,https://www.udemy.com/course/complete-python-b...,All Levels,Programming Languages,"(487,458 ratings)"
1,1,['Dr. Angela Yu'],The Complete 2023 Web Development Bootcamp,4.7,"1,137,534 students",Become a Full-Stack Web Developer with just ON...,English,Last updated 11/2023,https://www.udemy.com/course/the-complete-web-...,All Levels,Web Development,"(340,840 ratings)"
2,2,['Colt Steele'],The Web Developer Bootcamp 2023,4.7,"890,730 students",10 Hours of React just added. Become a Develop...,English,Last updated 11/2023,https://www.udemy.com/course/the-web-developer...,All Levels,Web Development,"(266,901 ratings)"
3,3,['Dr. Angela Yu'],100 Days of Code: The Complete Python Pro Boot...,4.7,"1,098,710 students",Master Python by building 100 projects in 100 ...,English,Last updated 11/2023,https://www.udemy.com/course/100-days-of-code/,All Levels,Programming Languages,"(254,397 ratings)"
4,4,"['Academind by Maximilian Schwarzmüller', 'Max...",React - The Complete Guide 2023 (incl. React R...,4.6,"807,714 students",Dive in and learn React.js from scratch! Learn...,English,Last updated 11/2023,https://www.udemy.com/course/react-the-complet...,All Levels,Programming Languages,"(200,267 ratings)"


In [50]:
courses_data.shape

(320, 12)

In [64]:
indices = pd.Series(courses_data.index, index=courses_data['course_title']).drop_duplicates()

In [65]:
indices

course_title
The Complete Python Bootcamp From Zero to Hero in Python          0
The Complete 2023 Web Development Bootcamp                        1
The Web Developer Bootcamp 2023                                   2
100 Days of Code: The Complete Python Pro Bootcamp for 2023       3
React - The Complete Guide 2023 (incl. React Router & Redux)      4
                                                               ... 
Ultimate Rust Crash Course                                      315
Microfrontends with React: A Complete Developer's Guide         316
Data Integration Fundamentals                                   317
Complete guide to building an app with .Net Core and React      318
Artificial Intelligence & Machine Learning for Business         319
Length: 320, dtype: int64

In [66]:
selected_features = ['course_title','descriptions','category']

In [67]:
for feature in selected_features:
  courses_data[feature] = courses_data[feature].fillna('')


In [68]:
combined_features = courses_data['course_title']+' '+courses_data['descriptions']+' '+courses_data['category']

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

In [70]:
tfv_matrix = tfv.fit_transform(combined_features)

In [71]:
tfv_matrix.shape


(320, 649)

In [72]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [73]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:11]

    # Movie indices
    course_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return courses_data['course_title'].iloc[course_indices]

In [76]:
give_rec('Artificial Intelligence & Machine Learning for Business')

301    Machine Learning for Absolute Beginners - Level 1
215    Tensorflow 2.0: Deep Learning and Artificial I...
112    Complete Machine Learning & Data Science Bootc...
227    Artificial Intelligence: Reinforcement Learnin...
312     The Complete Machine Learning Course with Python
55     Artificial Intelligence A-Z 2023: Build 5 AI (...
161    Introduction to Machine Learning for Data Science
67     Machine Learning, Data Science and Generative ...
9      Python for Data Science and Machine Learning B...
8      Machine Learning A-Z™: AI, Python & R + ChatGP...
Name: course_title, dtype: object